In [1]:
!pip install -U transformers

In [2]:
from datasets import load_dataset

# Example: English ↔ Hindi
dataset = load_dataset("opus100", "en-hi")


/Users/kaustubhkislay/miniconda3/envs/llmlearn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

df = pd.DataFrame(dataset['train']['translation'])
df.head()


,en,hi
0,"Other, Private Use","अन्य, निज़ी उपयोग"
1,[SCREAMING],ऊबड़ .
2,Spouse,जीवनसाथी
3,I will never salute you!,- तुम एक कमांडर कभी नहीं होगा!
4,and the stars and the trees bow themselves;,और तारे और वृक्ष सजदा करते है;


In [4]:
small_df = df.sample(5000, random_state=42)


In [5]:
small_df = small_df[(small_df['en'].str.len() < 200) & (small_df['hi'].str.len() < 200)]


In [6]:
small_df.to_json("/Users/kaustubhkislay/LLM-Language-Learning/data/OPUS/OPUSen_hi_opus.json", orient="records", lines=True)


In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b")



In [8]:
MAX_LENGTH = 128  # adjust based on your study design

def filter_long_sentences(example):
    return len(example["en"].split()) < MAX_LENGTH and \
           len(example["hi"].split()) < MAX_LENGTH

In [9]:


tokenized = dataset["train"].map(lambda x: {
    "en": x["translation"]["en"],
    "hi": x["translation"]["hi"]
})
tokenized = tokenized.filter(filter_long_sentences)


In [10]:
def tokenize_function(examples):
    return tokenizer(examples["en"], text_target=examples["hi"])

tokenized = tokenized.map(tokenize_function, batched=True)


In [11]:
tokenized_split = tokenized.train_test_split(test_size=0.1, seed=42)
train_dataset = tokenized_split['train']
eval_dataset  = tokenized_split['test']

In [12]:
tokenized_split = tokenized.train_test_split(test_size=0.1, seed=42)
train_dataset = tokenized_split['train']
eval_dataset  = tokenized_split['test']

In [13]:
import torch
from transformers import AutoModelForCausalLM

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-1.4b").to(device)


In [14]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="/Users/kaustubhkislay/LLM-Language-Learning/checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    save_strategy="epoch",
    logging_dir="./logs",
    eval_strategy="epoch",
    learning_rate=5e-5,
)

/Users/kaustubhkislay/miniconda3/envs/llmlearn/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/Users/kaustubhkislay/miniconda3/envs/llmlearn/lib/python3.10/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. Th

In [15]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [16]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_split["train"],
    eval_dataset=tokenized_split["test"],
    data_collator=data_collator,
)

In [18]:
trainer.train()

wandb: Currently logged in as: kaustubh-kislay (kaustubh-kislay-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/Users/kaustubhkislay/miniconda3/envs/llmlearn/lib/python3.10/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


RuntimeError: MPS backend out of memory (MPS allocated: 19.99 GiB, other allocations: 395.89 MiB, max allowed: 20.40 GiB). Tried to allocate 48.00 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).